#  Pandas - Remodelagem e tabelas dinâmicas ( stack, unstack, melt, wide_to_long)

In [86]:
import numpy as np
import pandas as pd
import os

diratual = os.getcwd()

diretorio = os.path.join(diratual) 

In [87]:
dfcovid = pd.read_csv(os.path.join(diretorio, 'HIST_PAINEL_COVIDBR_02nov2024.csv') , sep=';')
dfcovid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0.0,0,0,0,0.0,0.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1.0,1,0,0,1.0,0.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1.0,0,0,0,1.0,0.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1.0,0,0,0,0.0,1.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2.0,1,0,0,1.0,1.0,NaN


In [88]:
filtro = (dfcovid['estado'].isna() == False ) & ( dfcovid['codmun'].isna()  )
dfcovid = dfcovid[filtro][['regiao', 'estado', 'data' , 'populacaoTCU2019',
       'casosNovos', 'obitosNovos']]

dfcovid['mes'] = dfcovid['data'].map(lambda x : x[:7])


print(dfcovid.shape)
dfcovid.head()

(46251, 7)


,regiao,estado,data,populacaoTCU2019,casosNovos,obitosNovos,mes
158,Norte,RO,2020-02-25,1777225.0,0,0,2020-02
159,Norte,RO,2020-02-26,1777225.0,0,0,2020-02
160,Norte,RO,2020-02-27,1777225.0,0,0,2020-02
161,Norte,RO,2020-02-28,1777225.0,0,0,2020-02
162,Norte,RO,2020-02-29,1777225.0,0,0,2020-02


In [89]:
dfregiaouf = dfcovid[['regiao', 'estado', 'mes',  'casosNovos', 'obitosNovos', 'populacaoTCU2019', 'data']]\
  .groupby(['regiao', 'estado', 'mes', 'data' ])\
  .agg({'casosNovos':'sum', 'obitosNovos':'sum', 'populacaoTCU2019':'max'})\
  .reset_index()
dfregiaouf.head()

,regiao,estado,mes,data,casosNovos,obitosNovos,populacaoTCU2019
0,Centro-Oeste,DF,2020-02,2020-02-25,0,0,3015268.0
1,Centro-Oeste,DF,2020-02,2020-02-26,0,0,3015268.0
2,Centro-Oeste,DF,2020-02,2020-02-27,0,0,3015268.0
3,Centro-Oeste,DF,2020-02,2020-02-28,0,0,3015268.0
4,Centro-Oeste,DF,2020-02,2020-02-29,0,0,3015268.0


In [90]:
dfregiaouf.columns

Index(['regiao', 'estado', 'mes', 'data', 'casosNovos', 'obitosNovos',
       'populacaoTCU2019'],
      dtype='object')

In [91]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='index',
                      values=dfregiaouf['obitosNovos'], aggfunc="sum",
                      margins=True, margins_name = 'Total')
print(dfresumo.loc['Total'].sum())
dfresumo.head(30)

1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.024876,0.109453,0.019900,0.800995,0.044776
2020-04,0.012982,0.265614,0.124211,0.567193,0.030000
2020-05,0.012643,0.313074,0.212617,0.445906,0.015760
2020-06,0.044749,0.343857,0.126684,0.449868,0.034841
2020-07,0.109182,0.284085,0.070679,0.435875,0.100179
2020-08,0.135197,0.224901,0.056251,0.442261,0.141389
2020-09,0.150990,0.180630,0.069337,0.462629,0.136414
2020-10,0.137459,0.187045,0.058499,0.487509,0.129488


## Remodelando com  empilhamento e  desempilhamento  

O Pandas possui métodos stack()e unstack() relacionados ao pivot(), que para dinamizar para trabalhar em conjunto com MultiIndex.

- stack: “Dinamizar” um nível dos rótulos de coluna (possivelmente hierárquicos), retornando a DataFrame com um índice com um novo nível mais interno de rótulos de linha.


- unstack: (operação inversa de stack) “dinamiza” um nível do índice de linha (possivelmente hierárquico) para o eixo da coluna, produzindo um remodelado DataFramecom um novo nível mais interno de rótulos de coluna.


- Se as colunas tiverem um MultiIndex, pode-se escolher qual nível empilhar. O nível empilhado se torna o novo nível mais baixo em a MultiIndexnas colunas.


- A operação inversa de  unstack por padrão descompacta o último nível.


- Se os índices tiverem nomes, pode-se usar os nomes dos níveis em vez de especificar os números dos níveis

In [92]:
dfresumo = pd.crosstab(dfregiaouf['mes'], [dfregiaouf['regiao'], dfregiaouf['estado']], 
                       rownames=['Meses'], colnames=['Região', 'Estados'],
                       values=dfregiaouf['obitosNovos'], aggfunc="sum")
dfresumo.head(28)

Região  Centro-Oeste                   Nordeste                                \
Estados           DF    GO    MS    MT       AL    BA    CE    MA    PB    PE   
Meses                                                                           
2020-02            0     0     0     0        0     0     0     0     0     0   
2020-03            3     1     1     0        1     2     7     1     0     6   
2020-04           27    28     8    11       46   102   475   183    62   559   
2020-05          140    95    11    50      396   563  2528   771   298  2242   
2020-06          417   350    56   532      609  1186  3136  1093   617  2022   
2020-07          882  1182   300  1226      515  1610  1522   965   834  1728   
2020-08         1052  1438   486   932      320  1934   741   433   639  1036   
2020-09          734  1578   441   655      185  1347   585   310   374   658   
2020-10          428  1081   289   392      166   878   359   297   277   376   
2020-11          247   598   177   287      101   646   259   243   194   410   
2020-12          329   454   560   370      150   861   378   204   377   617   
2021-01          295   683   580   595      257   968   486   197   384   694   
2021-02          284  1029   410   663      253  1722   808   355   440   648   
2021-03         1191  3081   958  1829      555  3511  2740  1018  1248  1179   
2021-04         1769  3402  1441  2071      666  3147  3394  1206  1052  1863   
2021-05          872  2033  1097  1135      531  2764  3056   864   876  1824   
2021-06          581  2154  1389  1069      589  2771  2097   903   934  1823   
2021-07          369  1577   734   732      469  1738   948   593   381  1099   
2021-08          434  1664   432   689      263   737   496   395   200   599   
2021-09          410  1065   185   325      141   372   206   143   124   357   
2021-10          413   705    91   118       79   218   276    63   111   278   
2021-11          158   317    38    68       60   232   162    57   107   225   
2021-12           73   163    44    48       31   197   147    83    67   204   
2022-01           66   359   150   247       58   438   439   111   120   196   
2022-02          242   704   450   391      243  1237  1060   302   376   433   
2022-03          168   501   174   154      200   532   426    82   101   335   
2022-04           67   232    26    26       42   143   176    11    16   189   
2022-05           40   181    41    20       10    82   175     6    11   124   

Região   ... Norte           Sudeste                      Sul              
Estados  ...    RO   RR   TO      ES    MG    RJ     SP    PR    RS    SC  
Meses    ...                                                               
2020-02  ...     0    0    0       0     0     0      0     0     0     0  
2020-03  ...     1    0    0       0     2    23    136     3     4     2  
2020-04  ...    15    7    3      83    80   831   2239    80    47    44  
2020-05  ...   140  109   70     521   189  4490   5240    99   173    97  
2020-06  ...   362  167  127    1044   694  4736   7148   467   390   198  
2020-07  ...   354  222  181     897  1804  3397   8234  1271  1262   761  
2020-08  ...   276   82  292     613  2566  2588   7017  1370  1559  1158  
2020-09  ...   209   61  266     387  2025  2422   5608  1195  1347   537  
2020-10  ...   100   44  158     310  1655  2113   3689   731  1015   317  
2020-11  ...   104   35   67     417  1026  1990   2784   938  1016   648  
2020-12  ...   256   54   70     808  1861  2940   4622  1820  2059  1491  
2021-01  ...   427   75  148     777  3158  4281   6317  2056  1797  1087  
2021-02  ...   606  244  144     549  3505  3269   6459  1639  1723  1018  
2021-03  ...  1293  241  506    1084  5767  3647  15159  5048  7344  3527  
2021-04  ...  1020  167  514    2027  9367  7679  21539  5680  5215  2649  
2021-05  ...   595  127  331    1273  6798  6178  15183  4024  3241  1742  
2021-06  ...   378  110  336     685  5745  

In [93]:
dfregistros =  dfresumo.stack(future_stack=True)
dfregistros.tail()

Região           Centro-Oeste  Nordeste  Norte  Sudeste   Sul
Meses   Estados                                              
2024-11 RJ                NaN       NaN    NaN     12.0   NaN
        SP                NaN       NaN    NaN     50.0   NaN
        PR                NaN       NaN    NaN      NaN   8.0
        RS                NaN       NaN    NaN      NaN  17.0
        SC                NaN       NaN    NaN      NaN   1.0

In [94]:
dfregistros =  dfresumo.copy()
dfregistros.columns = [ x[1] for x in dfregistros.columns ]
dfregistros.tail()

,DF,GO,MS,MT,AL,BA,CE,MA,PB,PE,...,RO,RR,TO,ES,MG,RJ,SP,PR,RS,SC
Meses,,,,,,,,,,,,,,,,,,,,,
2024-07,3,6,2,1,3,1,0,2,3,3,...,1,0,0,0,118,17,55,1,8,-1
2024-08,2,12,16,9,6,9,0,1,6,4,...,2,0,8,1,21,18,192,-6,8,4
2024-09,5,38,27,0,1,2,0,0,5,1,...,2,0,1,0,49,22,214,75,11,7
2024-10,6,27,10,12,1,16,0,0,2,0,...,0,0,5,2,97,22,122,6,14,18
2024-11,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,19,12,50,8,17,1


In [95]:
dfregistros =  dfregistros.stack()
dfregistros.reset_index().tail()

,Meses,level_1,0
1561,2024-11,RJ,12
1562,2024-11,SP,50
1563,2024-11,PR,8
1564,2024-11,RS,17
1565,2024-11,SC,1


In [96]:
dfresumo.head()

Região  Centro-Oeste               Nordeste                               ...  \
Estados           DF   GO  MS   MT       AL    BA    CE    MA   PB    PE  ...   
Meses                                                                     ...   
2020-02            0    0   0    0        0     0     0     0    0     0  ...   
2020-03            3    1   1    0        1     2     7     1    0     6  ...   
2020-04           27   28   8   11       46   102   475   183   62   559  ...   
2020-05          140   95  11   50      396   563  2528   771  298  2242  ...   
2020-06          417  350  56  532      609  1186  3136  1093  617  2022  ...   

Região  Norte           Sudeste                   Sul            
Estados    RO   RR   TO      ES   MG    RJ    SP   PR   RS   SC  
Meses                                                            
2020-02     0    0    0       0    0     0     0    0    0    0  
2020-03     1    0    0       0    2    23   136    3    4    2  
2020-04    15    7    3      83   80   831  2239   80   47   44  
2020-05   140  109   70     521  189  4490  5240   99  173   97  
2020-06   362  167  127    1044  694  4736  7148  467  390  198  

[5 rows x 27 columns]

In [97]:
dfregistros =  dfresumo.stack(1,future_stack=True)
dfregistros.tail()

Região           Centro-Oeste  Nordeste  Norte  Sudeste   Sul
Meses   Estados                                              
2024-11 RJ                NaN       NaN    NaN     12.0   NaN
        SP                NaN       NaN    NaN     50.0   NaN
        PR                NaN       NaN    NaN      NaN   8.0
        RS                NaN       NaN    NaN      NaN  17.0
        SC                NaN       NaN    NaN      NaN   1.0

In [98]:
dfregistros =  dfresumo.stack(['Região', 'Estados'],future_stack=True).reset_index()
dfregistros.head()

,Meses,Região,Estados,0
0,2020-02,Centro-Oeste,DF,0
1,2020-02,Centro-Oeste,GO,0
2,2020-02,Centro-Oeste,MS,0
3,2020-02,Centro-Oeste,MT,0
4,2020-02,Nordeste,AL,0


In [99]:
dfregistros =  dfresumo.stack([0,1],future_stack=True).reset_index()
dfregistros.head()

,Meses,Região,Estados,0
0,2020-02,Centro-Oeste,DF,0
1,2020-02,Centro-Oeste,GO,0
2,2020-02,Centro-Oeste,MS,0
3,2020-02,Centro-Oeste,MT,0
4,2020-02,Nordeste,AL,0


In [100]:
dfregistros =  dfresumo.stack(['Região', 'Estados'],future_stack=True).unstack('Meses')
dfregistros.head()

Meses                 2020-02  2020-03  2020-04  2020-05  2020-06  2020-07  \
Região       Estados                                                         
Centro-Oeste DF             0        3       27      140      417      882   
             GO             0        1       28       95      350     1182   
             MS             0        1        8       11       56      300   
             MT             0        0       11       50      532     1226   
Nordeste     AL             0        1       46      396      609      515   

Meses                 2020-08  2020-09  2020-10  2020-11  ...  2024-02  \
Região       Estados                                      ...            
Centro-Oeste DF          1052      734      428      247  ...       20   
             GO          1438     1578     1081      598  ...       24   
             MS           486      441      289      177  ...       12   
             MT           932      655      392      287  ...       32   
Nordeste     AL           320      185      166      101  ...        8   

Meses                 2024-03  2024-04  2024-05  2024-06  2024-07  2024-08  \
Região       Estados                                                         
Centro-Oeste DF            23        9        3        2        3        2   
             GO            57        7       10        9        6       12   
             MS            32        5        5        2        2       16   
             MT            19        6        2        2        1        9   
Nordeste     AL             6        4        0        0        3        6   

Meses                 2024-09  2024-10  2024-11  
Região       Estados                             
Centro-Oeste DF             5        6        0  
             GO            38       27        0  
             MS            27       10        1  
             MT             0       12        0  
Nordeste     AL             1        1        0  

[5 rows x 58 columns]

In [101]:
dfregistros =  dfresumo.stack([0,1],future_stack=True).unstack(0)
dfregistros.head()

Meses                 2020-02  2020-03  2020-04  2020-05  2020-06  2020-07  \
Região       Estados                                                         
Centro-Oeste DF             0        3       27      140      417      882   
             GO             0        1       28       95      350     1182   
             MS             0        1        8       11       56      300   
             MT             0        0       11       50      532     1226   
Nordeste     AL             0        1       46      396      609      515   

Meses                 2020-08  2020-09  2020-10  2020-11  ...  2024-02  \
Região       Estados                                      ...            
Centro-Oeste DF          1052      734      428      247  ...       20   
             GO          1438     1578     1081      598  ...       24   
             MS           486      441      289      177  ...       12   
             MT           932      655      392      287  ...       32   
Nordeste     AL           320      185      166      101  ...        8   

Meses                 2024-03  2024-04  2024-05  2024-06  2024-07  2024-08  \
Região       Estados                                                         
Centro-Oeste DF            23        9        3        2        3        2   
             GO            57        7       10        9        6       12   
             MS            32        5        5        2        2       16   
             MT            19        6        2        2        1        9   
Nordeste     AL             6        4        0        0        3        6   

Meses                 2024-09  2024-10  2024-11  
Região       Estados                             
Centro-Oeste DF             5        6        0  
             GO            38       27        0  
             MS            27       10        1  
             MT             0       12        0  
Nordeste     AL             1        1        0  

[5 rows x 58 columns]

### O  unstack pode receber um argumento opcional fill_value, para especificar o valor dos dados vazios.

In [102]:
dfregistros =   dfresumo.stack(['Região', 'Estados'],future_stack=True).unstack(fill_value=0)
dfregistros.tail()

Estados               AC  AL  AM  AP  BA  CE  DF  ES  GO  MA  ...  PR  RJ  RN  \
Meses   Região                                                ...               
2024-11 Centro-Oeste   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   
        Nordeste       0   0   0   0   0   0   0   0   0   0  ...   0   0   0   
        Norte          0   0   3   0   0   0   0   0   0   0  ...   0   0   0   
        Sudeste        0   0   0   0   0   0   0   0   0   0  ...   0  12   0   
        Sul            0   0   0   0   0   0   0   0   0   0  ...   8   0   0   

Estados               RO  RR  RS  SC  SE  SP  TO  
Meses   Região                                    
2024-11 Centro-Oeste   0   0   0   0   0   0   0  
        Nordeste       0   0   0   0   1   0   0  
        Norte          0   0   0   0   0   0   0  
        Sudeste        0   0   0   0   0  50   0  
        Sul            0   0  17   1   0   0   0  

[5 rows x 27 columns]

## Remodelagem com a função  melt()

A função melt() é útil para transformar um DataFrame em um formato onde uma ou mais colunas são variáveis **identificadoras** , enquanto todas as outras colunas, consideradas variáveis **medidas** , são "não dinamizadas" para o eixo da linha, resultando apenas duas colunas não identificadoras , “Variável” e “valor”. 



In [103]:
dfresumo = pd.pivot_table(dfregiaouf, values=['obitosNovos'], 
               index=['mes'], columns=['estado'])
dfresumo.head(10)

obitosNovos                                                         \
estado           AC         AL         AM        AP         BA          CE   
mes                                                                          
2020-02    0.000000   0.000000   0.000000  0.000000   0.000000    0.000000   
2020-03    0.000000   0.032258   0.096774  0.000000   0.064516    0.225806   
2020-04    0.633333   1.533333  14.066667  1.133333   3.400000   15.833333   
2020-05    4.161290  12.774194  52.483871  6.064516  18.161290   81.548387   
2020-06    7.233333  20.300000  25.700000  6.500000  39.533333  104.533333   
2020-07    5.354839  16.612903  14.354839  4.774194  51.935484   49.096774   
2020-08    2.612903  10.322581  12.290323  3.096774  62.387097   23.903226   
2020-09    1.566667   6.166667  16.900000  1.600000  44.900000   19.500000   
2020-10    1.096774   5.354839  12.483871  1.258065  28.322581   11.580645   
2020-11    1.000000   3.366667  11.700000  1.966667  21.533333    8.633333   

                                                     ...             \
estado          DF         ES         GO         MA  ...         PR   
mes                                                  ...              
2020-02   0.000000   0.000000   0.000000   0.000000  ...   0.000000   
2020-03   0.096774   0.000000   0.032258   0.032258  ...   0.096774   
2020-04   0.900000   2.766667   0.933333   6.100000  ...   2.666667   
2020-05   4.516129  16.806452   3.064516  24.870968  ...   3.193548   
2020-06  13.900000  34.800000  11.666667  36.433333  ...  15.566667   
2020-07  28.451613  28.935484  38.129032  31.129032  ...  41.000000   
2020-08  33.935484  19.774194  46.387097  13.967742  ...  44.193548   
2020-09  24.466667  12.900000  52.600000  10.333333  ...  39.833333   
2020-10  13.806452  10.000000  34.870968   9.580645  ...  23.580645   
2020-11   8.233333  13.900000  19.933333   8.100000  ...  31.266667   

                                                                           \
estado           RJ         RN         RO        RR         RS         SC   
mes                                                                         
2020-02    0.000000   0.000000   0.000000  0.000000   0.000000   0.000000   
2020-03    0.741935   0.032258   0.032258  0.000000   0.129032   0.064516   
2020-04   27.700000   1.833333   0.500000  0.233333   1.566667   1.466667   
2020-05  144.838710   8.032258   4.516129  3.516129   5.580645   3.129032   
2020-06  157.866667  24.300000  12.066667  5.566667  13.000000   6.600000   
2020-07  109.580645  23.967742  11.419355  7.161290  40.709677  24.548387   
2020-08   83.483871  15.451613   8.903226  2.645161  50.290323  37.354839   
2020-09   80.733333   4.566667   6.966667  2.033333  44.900000  17.900000   
2020-10   68.161290   5.935484   3.225806  1.419355  32.741935  10.225806   
2020-11   66.333333   3.700000   3.466667  1.166667  33.866667  21.600000   

                                          
estado          SE          SP        TO  
mes                                       
2020-02   0.000000    0.000000  0.000000  
2020-03   0.000000    4.387097  0.000000  
2020-04   0.400000   74.633333  0.100000  
2020-05   4.709677  169.032258  2.258065  
2020-06  17.266667  238.266667  4.233333  
2020-07  24.451613  265.612903  5.838710  
2020-08  13.645161  226.354839  9.419355  
2020-09   5.966667  186.933333  8.866667  
2020-10   5.483871  119.000000  5.096774  
2020-11   3.400000   92.800000  2.233333  

[10 rows x 27 columns]

In [104]:
varscolumns = [c for _ , c in  dfresumo.columns]
print(varscolumns)

dfresumo.columns = varscolumns
dfresumo.reset_index(inplace=True)
dfresumo.head()

['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']


,mes,AC,AL,AM,AP,BA,CE,DF,ES,GO,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,2020-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-03,0.000000,0.032258,0.096774,0.000000,0.064516,0.225806,0.096774,0.000000,0.032258,...,0.096774,0.741935,0.032258,0.032258,0.000000,0.129032,0.064516,0.000000,4.387097,0.000000
2,2020-04,0.633333,1.533333,14.066667,1.133333,3.400000,15.833333,0.900000,2.766667,0.933333,...,2.666667,27.700000,1.833333,0.500000,0.233333,1.566667,1.466667,0.400000,74.633333,0.100000
3,2020-05,4.161290,12.774194,52.483871,6.064516,18.161290,81.548387,4.516129,16.806452,3.064516,...,3.193548,144.838710,8.032258,4.516129,3.516129,5.580645,3.129032,4.709677,169.032258,2.258065
4,2020-06,7.233333,20.300000,25.700000,6.500000,39.533333,104.533333,13.900000,34.800000,11.666667,...,15.566667,157.866667,24.300000,12.066667,5.566667,13.000000,6.600000,17.266667,238.266667,4.233333


In [105]:
dfresumo.head()

,mes,AC,AL,AM,AP,BA,CE,DF,ES,GO,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,2020-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-03,0.000000,0.032258,0.096774,0.000000,0.064516,0.225806,0.096774,0.000000,0.032258,...,0.096774,0.741935,0.032258,0.032258,0.000000,0.129032,0.064516,0.000000,4.387097,0.000000
2,2020-04,0.633333,1.533333,14.066667,1.133333,3.400000,15.833333,0.900000,2.766667,0.933333,...,2.666667,27.700000,1.833333,0.500000,0.233333,1.566667,1.466667,0.400000,74.633333,0.100000
3,2020-05,4.161290,12.774194,52.483871,6.064516,18.161290,81.548387,4.516129,16.806452,3.064516,...,3.193548,144.838710,8.032258,4.516129,3.516129,5.580645,3.129032,4.709677,169.032258,2.258065
4,2020-06,7.233333,20.300000,25.700000,6.500000,39.533333,104.533333,13.900000,34.800000,11.666667,...,15.566667,157.866667,24.300000,12.066667,5.566667,13.000000,6.600000,17.266667,238.266667,4.233333


In [106]:
dfresumo.melt(id_vars=['mes']).head(10)

,mes,variable,value
0,2020-02,AC,0.000000
1,2020-03,AC,0.000000
2,2020-04,AC,0.633333
3,2020-05,AC,4.161290
4,2020-06,AC,7.233333
5,2020-07,AC,5.354839
6,2020-08,AC,2.612903
7,2020-09,AC,1.566667
8,2020-10,AC,1.096774
9,2020-11,AC,1.000000


Os nomes dessas duas colunas podem ser personalizados fornecendo os parâmetros var_namee value_name.

In [107]:
dfresumo.melt(id_vars=['mes'], var_name='estado', value_name='obitos').head(10)

,mes,estado,obitos
0,2020-02,AC,0.000000
1,2020-03,AC,0.000000
2,2020-04,AC,0.633333
3,2020-05,AC,4.161290
4,2020-06,AC,7.233333
5,2020-07,AC,5.354839
6,2020-08,AC,2.612903
7,2020-09,AC,1.566667
8,2020-10,AC,1.096774
9,2020-11,AC,1.000000


Os valores do índice original podem ser mantidos indicando o parâmetro ignore_index como False(o padrão é True), mas isso irá duplicá-los.

In [108]:
dfmelt = dfresumo.melt(id_vars=['mes'], var_name='estado', value_name='obitos',  ignore_index=False)
dfmelt.head(10)

,mes,estado,obitos
0,2020-02,AC,0.000000
1,2020-03,AC,0.000000
2,2020-04,AC,0.633333
3,2020-05,AC,4.161290
4,2020-06,AC,7.233333
5,2020-07,AC,5.354839
6,2020-08,AC,2.612903
7,2020-09,AC,1.566667
8,2020-10,AC,1.096774
9,2020-11,AC,1.000000


In [109]:
dfmelt.loc[1].head()

,mes,estado,obitos
1,2020-03,AC,0.000000
1,2020-03,AL,0.032258
1,2020-03,AM,0.096774
1,2020-03,AP,0.000000
1,2020-03,BA,0.064516


## Remodelagem com a função wide_to_long()

Outra maneira de transformar é usar a função wide_to_long() que é menos flexível melt(), mas mais amigável.

## Remodelagem com a função wide_to_long()

Outra maneira de transformar é usar a função wide_to_long() que é menos flexível melt(), mas mais amigável.

In [110]:
dfresumo = pd.pivot_table(dfregiaouf, values=['obitosNovos'], 
               index=['regiao','estado'], columns=['mes'])
varscolumns = [c for _ , c in  dfresumo.columns]
print(varscolumns)

dfresumo.columns = varscolumns
dfresumo.reset_index(inplace=True)
dfresumo.head()

['2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11']


,regiao,estado,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,...,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
0,Centro-Oeste,DF,0.0,0.096774,0.900000,4.516129,13.900000,28.451613,33.935484,24.466667,...,0.689655,0.741935,0.300000,0.096774,0.066667,0.096774,0.064516,0.166667,0.193548,0.0
1,Centro-Oeste,GO,0.0,0.032258,0.933333,3.064516,11.666667,38.129032,46.387097,52.600000,...,0.827586,1.838710,0.233333,0.322581,0.300000,0.193548,0.387097,1.266667,0.870968,0.0
2,Centro-Oeste,MS,0.0,0.032258,0.266667,0.354839,1.866667,9.677419,15.677419,14.700000,...,0.413793,1.032258,0.166667,0.161290,0.066667,0.064516,0.516129,0.900000,0.322581,0.5
3,Centro-Oeste,MT,0.0,0.000000,0.366667,1.612903,17.733333,39.548387,30.064516,21.833333,...,1.103448,0.612903,0.200000,0.064516,0.066667,0.032258,0.290323,0.000000,0.387097,0.0
4,Nordeste,AL,0.0,0.032258,1.533333,12.774194,20.300000,16.612903,10.322581,6.166667,...,0.275862,0.193548,0.133333,0.000000,0.000000,0.096774,0.193548,0.033333,0.032258,0.0


In [111]:
pd.wide_to_long(dfresumo, stubnames=['2020','2021','2022','2023','2024'], sep='-', i=['regiao','estado'], j="Mês").reset_index()

,regiao,estado,Mês,2020,2021,2022,2023,2024
0,Centro-Oeste,DF,2,0.000000,10.142857,8.642857,0.214286,0.689655
1,Centro-Oeste,DF,3,0.096774,38.419355,5.419355,0.290323,0.741935
2,Centro-Oeste,DF,4,0.900000,58.966667,2.233333,0.100000,0.300000
3,Centro-Oeste,DF,5,4.516129,28.129032,1.290323,0.129032,0.096774
4,Centro-Oeste,DF,6,13.900000,19.366667,2.266667,0.066667,0.066667
...,...,...,...,...,...,...,...,...
319,Sul,SC,9,17.900000,19.100000,1.466667,1.333333,0.233333
320,Sul,SC,10,10.225806,12.483871,0.193548,0.387097,0.580645
321,Sul,SC,11,21.600000,10.966667,1.733333,1.500000,0.500000
322,Sul,SC,12,48.096774,6.258065,3.935484,1.032258,NaN


## Combinando com estatísticas e GroupBy 

- A combinação de pivot/ stack/ unstack com GroupBy e as funções estatísticas básicas de Series e DataFrame podem produzir algumas manipulações de dados muito expressivas e rápidas.

In [112]:
dfresumo = pd.pivot_table(dfregiaouf, values=['obitosNovos'], 
               columns=['regiao','estado'], index=['mes'])
dfresumo.head()

obitosNovos                                                        \
regiao  Centro-Oeste                                   Nordeste              
estado            DF         GO        MS         MT         AL         BA   
mes                                                                          
2020-02     0.000000   0.000000  0.000000   0.000000   0.000000   0.000000   
2020-03     0.096774   0.032258  0.032258   0.000000   0.032258   0.064516   
2020-04     0.900000   0.933333  0.266667   0.366667   1.533333   3.400000   
2020-05     4.516129   3.064516  0.354839   1.612903  12.774194  18.161290   
2020-06    13.900000  11.666667  1.866667  17.733333  20.300000  39.533333   

                                                      ...             \
regiao                                                ...      Norte   
estado           CE         MA         PB         PE  ...         RO   
mes                                                   ...              
2020-02    0.000000   0.000000   0.000000   0.000000  ...   0.000000   
2020-03    0.225806   0.032258   0.000000   0.193548  ...   0.032258   
2020-04   15.833333   6.100000   2.066667  18.633333  ...   0.500000   
2020-05   81.548387  24.870968   9.612903  72.322581  ...   4.516129   
2020-06  104.533333  36.433333  20.566667  67.400000  ...  12.066667   

                                                                           \
regiao                         Sudeste                                      
estado         RR        TO         ES         MG          RJ          SP   
mes                                                                         
2020-02  0.000000  0.000000   0.000000   0.000000    0.000000    0.000000   
2020-03  0.000000  0.000000   0.000000   0.064516    0.741935    4.387097   
2020-04  0.233333  0.100000   2.766667   2.666667   27.700000   74.633333   
2020-05  3.516129  2.258065  16.806452   6.096774  144.838710  169.032258   
2020-06  5.566667  4.233333  34.800000  23.133333  157.866667  238.266667   

                                         
regiao         Sul                       
estado          PR         RS        SC  
mes                                      
2020-02   0.000000   0.000000  0.000000  
2020-03   0.096774   0.129032  0.064516  
2020-04   2.666667   1.566667  1.466667  
2020-05   3.193548   5.580645  3.129032  
2020-06  15.566667  13.000000  6.600000  

[5 rows x 27 columns]

In [119]:
dfresumo.stack(future_stack=True).head()

obitosNovos                           
regiao         Centro-Oeste Nordeste Norte Sudeste Sul
mes     estado                                        
2020-02 DF              0.0      NaN   NaN     NaN NaN
        GO              0.0      NaN   NaN     NaN NaN
        MS              0.0      NaN   NaN     NaN NaN
        MT              0.0      NaN   NaN     NaN NaN
        AL              NaN      0.0   NaN     NaN NaN

In [120]:
dfresumo.stack(future_stack=True).sum()

             regiao      
obitosNovos  Centro-Oeste     2210.759598
             Nordeste         4504.837767
             Norte            1721.218001
             Sudeste         11378.299028
             Sul              3731.799319
dtype: float64

In [121]:
dfresumo.stack(future_stack=True).sum().unstack()

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
obitosNovos,2210.759598,4504.837767,1721.218001,11378.299028,3731.799319


In [122]:
dfresumo.stack(future_stack=True).sum().unstack().sum(1)

obitosNovos    23546.913714
dtype: float64

In [149]:
dfresumo.T.groupby(level=1).sum().T.sum()

regiao
Centro-Oeste     2210.759598
Nordeste         4504.837767
Norte            1721.218001
Sudeste         11378.299028
Sul              3731.799319
dtype: float64

In [152]:
for g, data in dfresumo.T.groupby(level=1):
    print(g, data.T)

Centro-Oeste          obitosNovos                                  
regiao  Centro-Oeste                                  
estado            DF          GO         MS         MT
mes                                                   
2020-02     0.000000    0.000000   0.000000   0.000000
2020-03     0.096774    0.032258   0.032258   0.000000
2020-04     0.900000    0.933333   0.266667   0.366667
2020-05     4.516129    3.064516   0.354839   1.612903
2020-06    13.900000   11.666667   1.866667  17.733333
2020-07    28.451613   38.129032   9.677419  39.548387
2020-08    33.935484   46.387097  15.677419  30.064516
2020-09    24.466667   52.600000  14.700000  21.833333
2020-10    13.806452   34.870968   9.322581  12.645161
2020-11     8.233333   19.933333   5.900000   9.566667
2020-12    10.612903   14.645161  18.064516  11.935484
2021-01     9.516129   22.032258  18.709677  19.193548
2021-02    10.142857   36.750000  14.642857  23.678571
2021-03    38.419355   99.387097  30.903226  59.0000